In [8]:
import pandas as pd
import glob
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")


In [13]:
# путь к папке
folder = "/Users/george/Documents/GitHub/manifold_learning/data"

# ищем все csv-файлы
all_csv = glob.glob(os.path.join(folder, "*.csv"))

# читаем и объединяем
df = pd.concat((pd.read_csv(f) for f in all_csv), ignore_index=True)
print(f"Считано файлов: {len(all_csv)}")

Считано файлов: 4485


In [14]:
df = df.rename(columns={"BBLS_OIL_COND":"oil", "MCF_GAS": 'gas', "BBLS_WTR":"water", "API_WellNo":"well_name", "RptDate":"date"})
df["date"] = pd.to_datetime(df["date"])
# важно, чтобы внутри каждой скважины ряды шли строго по времени
df = df.sort_values(by=["well_name", "date"]).reset_index(drop=True)

df = df[(df['oil'] >= 0) & (df['gas'] >= 0) & (df['water'] >= 0)]

# ресемплинг
df = (
    df.set_index("date")
      .groupby("well_name")
      .resample("M").sum(numeric_only=True)
      .reset_index()
)

In [17]:
# стандартный метод через PCA и KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score
from dateutil.relativedelta import relativedelta

# -------------------
# ПАРАМЕТРЫ
# -------------------
WELL_COL = "well_name"
DATE_COL = "date"
OIL_COL  = "oil"
GAS_COL  = "gas"
WTR_COL  = "water"
DAYS_COL = "DAYS_PROD"

# Сигнал для сегментации: 'bopd' (барр/сут из BBLS_OIL_COND/DAYS_PROD),
# 'bbl_mo' (месячные баррели), или 'mcfpd' (газ/сут) – выберите ниже:
SIGNAL = "bopd"           # 'bopd' | 'bbl_mo' | 'mcfpd'

# Старт профиля: первый месяц, когда ряд >= frac * (максимум внутри скважины)
START_THRESHOLD_FRAC = 0.05
MIN_CONSEC = 1

# Горизонт (кол-во месяцев после старта). Если None → возьмём минимально доступный у всех скважин.
HORIZON = 70  # поставьте 240, если у всех скважин 0..239

# Поиск числа кластеров
K_MIN, K_MAX = 1, 12
ISF_CONTAMINATION = 0.03
RANDOM_STATE = 42

OUT_DIR = Path("segm_long_output"); OUT_DIR.mkdir(exist_ok=True, parents=True)

# -------------------
# ВСПОМОГАТЕЛЬНОЕ
# -------------------
def monthly_signal_for_well(g: pd.DataFrame, signal_col: str) -> pd.Series:
    # g — все строки одной скважины
    g = g[[DATE_COL, OIL_COL, GAS_COL, WTR_COL, DAYS_COL] + [signal_col]].copy()
    g[DATE_COL] = pd.to_datetime(g[DATE_COL])

    # агрегируем на уровень календарного месяца
    if signal_col == "bopd":
        # средний суточный дебит нефти за месяц = (сумма баррелей) / (сумма дней)
        oil_sum  = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[OIL_COL].sum(min_count=1)
        days_sum = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[DAYS_COL].sum(min_count=1)
        s = oil_sum / days_sum
    elif signal_col == "mcfpd":
        gas_sum  = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[GAS_COL].sum(min_count=1)
        days_sum = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[DAYS_COL].sum(min_count=1)
        s = gas_sum / days_sum
    elif signal_col == "bbl_mo":
        s = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[OIL_COL].sum(min_count=1)
    else:
        # на всякий случай: среднее по месяцу
        s = g.groupby(pd.Grouper(key=DATE_COL, freq="ME"))[signal_col].mean()

    # нормализуем индекс к последнему дню месяца
    s.index = s.index.to_period("M").to_timestamp("M")
    return s.astype(float)


def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # дата
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])
    # чистка отрицательных и бессмысленных
    for c in [OIL_COL, GAS_COL, WTR_COL, DAYS_COL]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
            if c != DAYS_COL:
                df.loc[df[c] < 0, c] = np.nan
    df.loc[df[DAYS_COL] <= 0, DAYS_COL] = np.nan
    # вычисляем суточные дебиты
    df["bopd"]   = df[OIL_COL] / df[DAYS_COL]
    df["mcfpd"]  = df[GAS_COL] / df[DAYS_COL]
    df["bbl_mo"] = df[OIL_COL]
    # уберём нереальные пики (клипы по верхнему перцентилю по каждой скважине)
    for col in ["bopd", "mcfpd", "bbl_mo"]:
        if col not in df.columns: 
            continue
        def _clip(g):
            q99 = g[col].quantile(0.995)
            g.loc[g[col] > q99, col] = np.nan
            return g
        df = df.groupby(WELL_COL, group_keys=False).apply(_clip)
    return df

def detect_start(series: pd.Series, thr_frac=START_THRESHOLD_FRAC, min_consec=MIN_CONSEC):
    s = series.copy().astype(float).replace([np.inf, -np.inf], np.nan).interpolate(limit_direction="both")
    if s.dropna().empty: 
        return None
    mmax = s.max()
    if not np.isfinite(mmax) or mmax <= 0:
        return None
    thr = mmax * thr_frac
    flags = s >= thr
    run = 0
    for i, f in enumerate(flags.values):
        run = run + 1 if f else 0
        if run >= min_consec:
            return series.index[i - min_consec + 1]
    return None

def build_wide_matrix(df: pd.DataFrame, signal_col: str, horizon: int | None):
    rows, wells, starts = [], [], []
    # упорядочим месяцы строго по календарю
    df = df.sort_values([WELL_COL, DATE_COL])
    for w, g in df.groupby(WELL_COL):
        s = monthly_signal_for_well(g, signal_col)     # <-- уникальный индекс по месяцам, без дублей
    
        if s.dropna().sum() == 0:
            continue
    
        # старт
        start_idx = detect_start(s, START_THRESHOLD_FRAC, MIN_CONSEC)
        if start_idx is None:
            continue
    
        g2 = s.loc[start_idx:].copy()
        if g2.empty:
            continue
    
        cal = pd.date_range(start_idx, g2.index.max(), freq="ME")
        g2 = g2.reindex(cal).interpolate(limit_direction="both")
    
        g2.index = pd.Index(range(len(g2)), name="month_rel")
        rows.append(g2)
        wells.append(w)
        starts.append(start_idx)
    if not rows:
        raise RuntimeError("После выравнивания не осталось ни одной скважины.")
    # соберём в огромную таблицу (разные длины допустимы)
    wide = pd.DataFrame(rows, index=wells).fillna(np.nan)
    # выберем горизонт
    if horizon is None:
        horizon = int(wide.columns.max()) + 1
    use_cols = [c for c in wide.columns if (isinstance(c, (int, np.integer)) and c < horizon)]
    wide = wide.reindex(columns=use_cols)
    # отфильтруем скважины, у которых не хватает горизонта
    enough = wide.notna().sum(axis=1) >= len(use_cols)
    wide = wide.loc[enough]
    wells = list(wide.index)
    starts = [s for w, s in zip(wells, starts) if w in wide.index]
    return wide.to_numpy(dtype=float), wells, starts, len(use_cols)

def zscore_rows(X: np.ndarray):
    mu = np.nanmean(X, axis=1, keepdims=True)
    sd = np.nanstd(X, axis=1, keepdims=True) + 1e-8
    return (X - mu) / sd

def pick_kmeans_auto(X, k_min=K_MIN, k_max=K_MAX):
    best, best_sil = None, -1
    sils = {}
    for k in range(k_min, k_max+1):
        km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=25).fit(X)
        if len(set(km.labels_)) <= 1: 
            continue
        s = silhouette_score(X, km.labels_)
        sils[k] = float(s)
        if s > best_sil:
            best, best_sil = km, s
    if best is None:
        best = KMeans(n_clusters=3, random_state=RANDOM_STATE, n_init=25).fit(X)
    return best, sils

# -------------------
# ОСНОВНОЙ ХОД
# -------------------
# 1) источник данных:
# (а) если у вас уже есть df:
# df = df
# (б) либо загрузите:
# df = pd.read_csv("your_file.csv")  # или read_parquet/read_excel

# >>> ЗДЕСЬ подставьте ваш df <<<
# df = pd.read_csv("your_file.csv")

# ----- подготовка
df = prep_df(df)

# выбор сигнала
signal_col = {"bopd":"bopd", "bbl_mo":"bbl_mo", "mcfpd":"mcfpd"}[SIGNAL]

# 2) wide-матрица (скважины × месяцы от старта)
X_orig, wells, start_dates, horizon_used = build_wide_matrix(df, signal_col, HORIZON)

print(f"Скважин после фильтра: {len(wells)}; горизонт: {horizon_used} мес; сигнал: {signal_col}")

# 3) нормировка по форме
X = zscore_rows(X_orig)

# 4) кластеризация + аутлаеры
kmeans, sils = pick_kmeans_auto(X, K_MIN, K_MAX)
labels = kmeans.predict(X)
isf = IsolationForest(n_estimators=300, contamination=ISF_CONTAMINATION,
                      random_state=RANDOM_STATE).fit(X)
outliers = (isf.predict(X) == -1)

print("Подбор K по силуэту:", sils)
print("Выбрано кластеров:", kmeans.n_clusters)

# 5) PCA-визуализация + доли объяснённой дисперсии
pca = PCA(n_components=2, random_state=RANDOM_STATE).fit(X)
Z = pca.transform(X)
evr = pca.explained_variance_ratio_
print(f"Explained variance: PC1={evr[0]*100:.1f}%, PC2={evr[1]*100:.1f}%")

plt.figure(figsize=(9,7))
for c in sorted(set(labels)):
    m = (labels==c) & (~outliers)
    plt.scatter(Z[m,0], Z[m,1], s=25, alpha=0.8, label=f"Кластер {c}")
plt.scatter(Z[outliers,0], Z[outliers,1], s=60, marker="x", label="Аутлаеры", c="k")
plt.title(f"PCA: {SIGNAL} | N={len(wells)} | K={kmeans.n_clusters}\n"
          f"PC1={evr[0]*100:.1f}%  PC2={evr[1]*100:.1f}%")
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.legend()
plt.tight_layout(); plt.savefig(OUT_DIR/"pca_scatter.png", dpi=160); plt.close()

# 6) средние кривые кластеров (в исходных единицах сигнала)
t = np.arange(horizon_used)
plt.figure(figsize=(10,7))
for c in sorted(set(labels)):
    Xc = X_orig[labels==c]
    if Xc.size==0: continue
    mean = np.nanmean(Xc, axis=0)
    p10  = np.nanpercentile(Xc, 10, axis=0)
    p90  = np.nanpercentile(Xc, 90, axis=0)
    plt.plot(t, mean, label=f"Кластер {c}")
    plt.fill_between(t, p10, p90, alpha=0.15)
plt.title(f"Средние профили (ед.: {SIGNAL})")
plt.xlabel("Месяц от старта"); plt.ylabel(SIGNAL)
plt.tight_layout(); plt.legend()
plt.savefig(OUT_DIR/"cluster_means.png", dpi=160); plt.close()

# 7) выгрузки
pd.DataFrame({
    "well": wells,
    "cluster": labels,
    "is_outlier": outliers,
    "start_date": start_dates
}).to_csv(OUT_DIR/"clustered_wells.csv", index=False, encoding="utf-8-sig")

print("Готово. Смотрите папку:", OUT_DIR.resolve())


Скважин после фильтра: 2032; горизонт: 70 мес; сигнал: bopd
Подбор K по силуэту: {2: 0.2476759902188048, 3: 0.15909158887806013, 4: 0.10343935487439704, 5: 0.09325364776183989, 6: 0.09466025699487236, 7: 0.0840743098180394, 8: 0.08379450470539342, 9: 0.06541837274385999, 10: 0.079764981268202, 11: 0.07614426063525631, 12: 0.07484974303088747}
Выбрано кластеров: 2
Explained variance: PC1=26.5%, PC2=13.4%
Готово. Смотрите папку: /Users/george/Documents/GitHub/manifold_learning/notebooks/segm_long_output
